<a href="https://colab.research.google.com/github/JaredLevi18/Stock-market-predictor./blob/main/model_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=1746f53dbbbded48fdb44939cc9cd1d2050542c5bbfcaf2bb43247c1c86cddec
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import pandas_ta as ta
import pandas as pd
import yfinance as yf

In [3]:
# donwload the data
company = input("Enter the company: ")
def get_data(company):
  # downloading the data
  data = yf.download(tickers=company, start="2012-01-01", end="2023-09-16")

  # adding indicators with the help of pandas_ta
  data['RSI'] = ta.rsi(data.Close, length=15)

  data['EMAF'] = ta.ema(data.Close, length=20)
  data['EMAM'] = ta.ema(data.Close, length=100)
  data['EMAS'] = ta.ema(data.Close, length=150)

  data['Target'] = data['Adj Close']-data.Open
  data['Target'] = data['Target'].shift(-1)

  data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]
  data['TargetNextClose'] = data['Adj Close'].shift(-1)

  # cleaning data
  data.dropna(inplace=True)
  data.reset_index(inplace=True)
  data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)

  dataset = data.iloc[:, 0:11]

  return dataset

pd.set_option('display.max_columns', None)
dataset = get_data(company)
dataset.head(20)

Enter the company: MSFT
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Adj Close,RSI,EMAF,EMAM,EMAS,Target,TargetClass,TargetNextClose
0,30.000000,30.110001,29.809999,24.168549,52.616542,29.638579,30.074739,30.378000,-5.651293,0,24.418707
1,30.070000,30.350000,29.870001,24.418707,55.463289,29.697762,30.078407,30.376437,-5.734808,0,24.475191
2,30.209999,30.469999,30.110001,24.475191,56.101364,29.757975,30.083390,30.375822,-5.777626,0,24.612373
3,30.389999,30.650000,30.299999,24.612373,57.679066,29.828644,30.091639,30.377467,-5.952187,0,24.547813
4,30.500000,30.620001,30.250000,24.547813,56.652477,29.884964,30.098141,30.378030,-5.826389,0,24.523611
5,30.350000,30.459999,30.160000,24.523611,56.250217,29.933062,30.103921,30.378189,-5.825125,0,24.474874
6,30.299999,30.389999,30.010000,24.474874,52.770940,29.951818,30.104437,30.374901,-5.578266,0,24.531734
7,30.110001,30.280001,30.020000,24.531734,53.598865,29.975455,30.106330,30.372585,-5.357126,0,25.002874
8,30.360001,30.940001,30.260000,25.002874,59.847402,30.052078,30.119670,30.377981,-5.819645,0,25.100355
9,30.920000,30.920000,30.590000,25.100355,61.011264,30.132833,30.135122,30.384895,-5.849615,0,24.970385


In [4]:
from pandas.core.interchange.dataframe_protocol import DataFrame
# normalizing data
dataset = dataset.to_numpy() # no longer a pandas.DataFrame, now is a numpy array.
class normalizer():
  def __init__(self) -> None:
    super().__init__()
    self.mean = None
    self.std = None

  def fit_transform(self, x):
    self.mean = np.mean(x, axis=(0), keepdims=True)
    self.std = np.std(x, axis=(0), keepdims=True)
    normalized_x = (x - self.mean) / self.std
    return normalized_x

  def denormalize_data(self, x):
    return (x * self.std) + self.mean

scaler = normalizer()
normalized_data = scaler.fit_transform(dataset)
normalized_data.shape

(2795, 11)

In [5]:
X = []

backcandels = 30
for j in range(8):
  X.append([])
  for i in range(backcandels, normalized_data.shape[0]):
    X[j].append(normalized_data[i-backcandels:i,j])

# move axis from 0 to position 2
X = np.moveaxis(X, [0], [2])

X, yi = np.array(X), np.array(normalized_data[backcandels:,-1])  # remove some of the data to match X shape
y = np.reshape(yi, (len(yi),1))
X.shape, y.shape

((2765, 30, 8), (2765, 1))

In [6]:
# generating val and training datasets
def split(X, y):
  spitlimit = int(len(X) * 0.9)
  # No longer a numpy.array, now a torch.tensor
  X_train, X_val = torch.from_numpy(X[:spitlimit]), torch.from_numpy(X[spitlimit:])
  y_train, y_val = torch.from_numpy(y[:spitlimit]), torch.from_numpy(y[spitlimit:])
  return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = split(X, y)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

(torch.Size([2488, 30, 8]),
 torch.Size([277, 30, 8]),
 torch.Size([2488, 1]),
 torch.Size([277, 1]))

In [6]:
class Model(nn.Module):
  def __init__(self, input_size, hidden_state, num_layers, output_size, dropout) -> None:
    super().__init__()
    self.hidden_state = hidden_state
    self.linear = nn.Linear(input_size, hidden_state)
    self.relu = nn.ReLU()
    self.lstm = nn.LSTM(hidden_state, hidden_size=self.hidden_state, num_layers=num_layers, batch_first=True)
    self.dropout = nn.Dropout(dropout)
    self.linear2 = nn.Linear(num_layers*hidden_state, output_size)
    self.init_weights()

  def init_weights(self):
